Importing the modules

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/fellow')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/total_jpg/total_segmim')

In [ ]:
import os
os.chdir('/content/drive/My Drive/total_jpg')

In [ ]:
import os
os.chdir('/content/drive/My Drive/fellow')

In [ ]:
import os
import numpy as np
import scipy.io
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import pandas as pd

DATASET_DIR = '/content/drive/MyDrive/total_jpg'
SEG_DIR = '/content/drive/MyDrive/total_jpg/total_segmim'
LABELS_PATH = '/content/drive/MyDrive/fellow/imagelabels.mat'
SPLIT_PATH = '/content/drive/MyDrive/fellow/setid.mat'
IMG_SIZE = 224
NUM_CLASSES = 102
BATCH_SIZE = 32

labels = scipy.io.loadmat(LABELS_PATH)['labels'][0] - 1
split = scipy.io.loadmat(SPLIT_PATH)
train_ids = split['trnid'][0] - 1
val_ids = split['valid'][0] - 1
test_ids = split['tstid'][0] - 1

def get_image_path(index):
    return os.path.join(DATASET_DIR, f'image_{index+1:05d}.jpg')

x_paths = [get_image_path(i) for i in range(len(labels))]
y_labels = labels

def load_image(path):
    img = cv2.imread(path)
    if img is None:
        print(f"Warning: Could not read {path}")
        return None
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img[..., ::-1]

x_data = [load_image(p) for p in x_paths]
valid_indices = [i for i, img in enumerate(x_data) if img is not None]
x_data = np.array([x_data[i] for i in valid_indices])
y_data = to_categorical(y_labels[valid_indices], num_classes=NUM_CLASSES)

valid_indices_set = set(valid_indices)
train_ids = np.array([i for i in train_ids if i in valid_indices_set])
val_ids = np.array([i for i in val_ids if i in valid_indices_set])
test_ids = np.array([i for i in test_ids if i in valid_indices_set])

x_train, y_train = x_data[train_ids], y_data[train_ids]
x_val, y_val = x_data[val_ids], y_data[val_ids]
x_test, y_test = x_data[test_ids], y_data[test_ids]

datagen = ImageDataGenerator(horizontal_flip=True, rotation_range=20, zoom_range=0.2)
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)
val_gen = ImageDataGenerator().flow(x_val, y_val, batch_size=BATCH_SIZE)

tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir='./logs')
model_ckpt = tf.keras.callbacks.ModelCheckpoint('flower_model.h5', save_best_only=True)

base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
x = GlobalAveragePooling2D()(base_model.output)
output = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5,
    callbacks=[tensorboard_cb, model_ckpt],
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen)
)

y_pred = model.predict(x_test)
y_true = np.argmax(y_test, axis=1)
y_pred_cls = np.argmax(y_pred, axis=1)

report = classification_report(y_true, y_pred_cls, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report.to_csv("per_class_accuracy.csv")

plt.figure(figsize=(20, 6))
df_report[:-3].sort_values('recall', ascending=False)['recall'].plot(kind='bar', color='skyblue')
plt.title('Per-Class Recall (Accuracy)')
plt.xlabel('Class Index')
plt.ylabel('Recall')
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("per_class_recall_bar_chart.png")
plt.show()

predictions_df = pd.DataFrame({
    'Image_Index': test_ids,
    'True_Label': y_true,
    'Predicted_Label': y_pred_cls
})
predictions_df.to_csv("test_predictions.csv", index=False)


def make_gradcam_heatmap(img_array, model, last_conv_layer_name="conv5_block3_out", pred_index=None):
    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(np.expand_dims(img_array, axis=0))
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_output)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    conv_output = conv_output[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_output), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def show_gradcam(image, heatmap, alpha=0.4):
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    color_map = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed = color_map * alpha + image
    plt.imshow(np.uint8(superimposed[..., ::-1]))
    plt.axis('off')
    plt.show()

NUM_SAMPLES = 5
for i in range(NUM_SAMPLES):
    idx = test_ids[i]
    img = load_image(get_image_path(idx))
    heatmap = make_gradcam_heatmap(img, model)
    print(f"Grad-CAM for test image index: {idx}")
    show_gradcam(img, heatmap)


sample_img = load_image(get_image_path(test_ids[0]))
heatmap = make_gradcam_heatmap(sample_img, model)
show_gradcam(sample_img, heatmap)


def show_segmentation(image_path, seg_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image: {image_path}")
        return
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

    if not os.path.exists(seg_path):
        print(f"Segmentation file does not exist: {seg_path}")
        return

    seg = cv2.imread(seg_path, cv2.IMREAD_GRAYSCALE)
    if seg is None:
        print(f"Could not read segmentation file: {seg_path}")
        return

    seg = cv2.resize(seg, (IMG_SIZE, IMG_SIZE))
    mask = cv2.applyColorMap(seg * 10, cv2.COLORMAP_JET)
    combined = cv2.addWeighted(img, 0.6, mask, 0.4, 0)
    plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()


seg_name = f"segmim_{test_ids[0]+1:05d}.jpg"
seg_path = os.path.join(SEG_DIR, seg_name)
show_segmentation(get_image_path(test_ids[0]), seg_path)


Output hidden; open in https://colab.research.google.com to view.